<a href="https://colab.research.google.com/github/mohak1/DDoS_Attack_Classification/blob/master/DDOS_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
 #%tensorflow_version 2.x
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import numpy as np
np.random.seed(0)

Using TensorFlow backend.


In [3]:
#loading dataset
import pandas as pd
x = pd.read_csv("/content/drive/My Drive/dos_dataset/clean_5.csv")
y = pd.read_csv("/content/drive/My Drive/dos_dataset/y.csv")
y=y['Label']
x=x.drop(['Label'], axis=1)

#normalizing the data
normalized_df=(x-x.mean())/x.std()

normalized_df=normalized_df.drop(['Unnamed: 0'], axis=1)
normalized_df=normalized_df.drop(['Unnamed: 0.1'], axis=1)
normalized_df=normalized_df.drop(['Unnamed: 0.1.1'], axis=1)
normalized_df=normalized_df.drop(['Flow Bytes/s'], axis=1)
normalized_df=normalized_df.drop([' Flow Packets/s'], axis=1)

y = pd.get_dummies(y,prefix=['Label'], drop_first=True)

normalized_df.head()

,Source Port,Destination IP,Destination Port,Protocol,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Fwd IAT Mean,Fwd IAT Std,Fwd PSH Flags,Fwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Variance,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,Idle Std,Inbound
0,1.395596,0.157586,-0.607399,-2.031188,-0.725421,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.178761,-0.175445,-0.178330,-0.168507,-0.090471,-1.335866,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933777,-0.945029,0.015085,-0.725421,-0.054584,-0.057299,-0.162793,0.19758
1,0.347887,0.157586,-1.625958,-2.031188,-0.733744,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.216151,-0.222876,-0.227006,-0.222546,-0.090471,-1.324001,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933085,-0.945029,0.015085,-0.733744,7.027636,-0.057299,-0.162793,0.19758
2,1.196809,0.157586,-0.560525,-2.031188,-0.725421,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.182709,-0.180450,-0.183461,-0.174204,-0.090471,-1.335864,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933777,-0.945029,0.015085,-0.725421,-0.054584,-0.057299,-0.162793,0.19758
3,0.892029,0.157586,-1.562081,-2.031188,-0.725421,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.186161,-0.184840,-0.187948,-0.179186,-0.090471,-1.335861,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933777,-0.945029,0.015085,-0.725421,-0.054584,-0.057299,-0.162793,0.19758
4,-0.914218,0.157586,0.169285,0.489985,-0.424394,-0.494002,-0.472024,-0.480570,-0.085259,-0.138378,-0.095478,-0.076871,-0.216208,-0.222934,-0.227005,-0.222546,-0.090471,-0.610670,-0.471885,-0.494875,-0.483551,-0.069737,-0.090471,-0.452542,-0.153371,-0.101473,-0.467788,-0.480570,-0.095478,-0.424394,-0.362285,-0.057771,-0.162793,0.19758


In [4]:
y = pd.read_csv("/content/drive/My Drive/dos_dataset/y.csv")
y=y['Label']
y.head()

0    2
1    2
2    2
3    2
4    2
Name: Label, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normalized_df, y, test_size=0.3, random_state=0)
y_train = np.array(y_train)

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1]))

print('xtrain={}, ytrain={}, xtest={}, ytest={}'.format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

xtrain=(872058, 34), ytrain=(872058,), xtest=(373740, 34), ytest=(373740,)


**XGBoost**

> with metrics



In [6]:
!pip install xgboost

In [0]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

import pickle
pickle.dump(model, open("/content/drive/My Drive/dos_dataset/final_xgb.pickle.dat", "wb"))

In [0]:
y_pred=model.predict(X_test)

In [10]:
y_train.shape

(872058,)

In [15]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('XGBoost Classifier')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

XGBoost Classifier
Accuracy =  89.29871033338685
Confusion Matrix =
 [[17136     0     0     0     0     0     0     0]
 [    1 56298     6     0  3467     0     0     1]
 [    0     0 59645    42    17     5     0     0]
 [    2     1    24 60168    22     0   199     0]
 [   11 34241     8    68 21904     1     0    16]
 [    1     0     0     2     1 59765     2     0]
 [    1     0     0  1621    11     0 58430    19]
 [    1     0     0    20    55     0   129   399]]
Recall = 0.8929871033338684
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     17136
           1       0.62      0.94      0.75     59773
           2       1.00      1.00      1.00     59709
           3       0.97      1.00      0.98     60416
           4       0.86      0.39      0.54     56249
           5       1.00      1.00      1.00     59771
           6       0.99      0.97      0.98     60082
           7       0.92      0.66   